In [11]:
# Install packages if running in Colab
# !pip install tensorflow keras xgboost

import pandas as pd
import numpy as np
import time
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, Input
from tensorflow.keras.optimizers import Adam

# Step 1: Load preprocessed data
data = pd.read_csv('/content/sample_data/drone_data_preprocessed.csv')
X = data.drop('Label', axis=1).values
y = data['Label'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Optional: Use 30% of data for faster training
X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42, stratify=y_train)
X_test, _, y_test, _ = train_test_split(X_test, y_test, train_size=0.3, random_state=42, stratify=y_test)

# Feature scaling for SVM and FNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 2: SVM (fast hyperparameters)
svm_param_grid = {
    'kernel': ['rbf'],
    'C': [1, 10],
    'gamma': ['scale'],
    'degree': [3]
}
svm = SVC(probability=True)
svm_search = RandomizedSearchCV(svm, param_distributions=svm_param_grid, n_iter=3, cv=2, verbose=2, n_jobs=-1)
start = time.time()
svm_search.fit(X_train_scaled, y_train)
end = time.time()
print(f"SVM trained in {end-start:.2f} seconds")
joblib.dump(svm_search, "svm_model_fast.pkl")

# Step 3: XGBoost (fast hyperparameters)
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}
xgb_search = RandomizedSearchCV(xgb_model, param_distributions=xgb_param_grid, n_iter=3, cv=2, verbose=2, n_jobs=-1)
start = time.time()
xgb_search.fit(X_train, y_train)
end = time.time()
print(f"XGBoost trained in {end-start:.2f} seconds")
joblib.dump(xgb_search, "xgb_model_fast.pkl")

# Step 4: Feedforward Neural Network (FNN)
def create_fnn(input_dim, hidden_layers=[64, 32], dropout_rate=0.2, activation='relu', lr=0.001):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    for units in hidden_layers:
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

fnn = create_fnn(X_train_scaled.shape[1])
start = time.time()
history_fnn = fnn.fit(X_train_scaled, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=2)
end = time.time()
print(f"FNN trained in {end-start:.2f} seconds")
fnn.save("fnn_model_fast.h5")

# Step 5: Sequence data for LSTM/1D-CNN
def create_sequences(X, y, time_steps=5):
    X_seq, y_seq = [], []
    for i in range(len(X)-time_steps):
        X_seq.append(X[i:(i+time_steps)])
        y_seq.append(y[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 5
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, time_steps)

# Step 6: LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(32, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
lstm_model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start = time.time()
history_lstm = lstm_model.fit(X_train_seq, y_train_seq, validation_split=0.2, epochs=20, batch_size=32, verbose=2)
end = time.time()
print(f"LSTM trained in {end-start:.2f} seconds")
lstm_model.save("lstm_model_fast.h5")

# Step 7: 1D-CNN
cnn_model = Sequential()
cnn_model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
cnn_model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

start = time.time()
history_cnn = cnn_model.fit(X_train_seq, y_train_seq, validation_split=0.2, epochs=20, batch_size=32, verbose=2)
end = time.time()
print(f"1D-CNN trained in {end-start:.2f} seconds")
cnn_model.save("cnn_model_fast.h5")

print("FAST VERSION: All models trained and saved successfully!")


Fitting 2 folds for each of 2 candidates, totalling 4 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


SVM trained in 54.63 seconds
Fitting 2 folds for each of 3 candidates, totalling 6 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [20:37:13] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost trained in 4.59 seconds
Epoch 1/20
525/525 - 2s - 5ms/step - accuracy: 0.6690 - loss: 0.7027 - val_accuracy: 0.8139 - val_loss: 0.4763
Epoch 2/20
525/525 - 1s - 2ms/step - accuracy: 0.8268 - loss: 0.3985 - val_accuracy: 0.8930 - val_loss: 0.2554
Epoch 3/20
525/525 - 1s - 2ms/step - accuracy: 0.8667 - loss: 0.2851 - val_accuracy: 0.9214 - val_loss: 0.2126
Epoch 4/20
525/525 - 1s - 2ms/step - accuracy: 0.8785 - loss: 0.2512 - val_accuracy: 0.9090 - val_loss: 0.2020
Epoch 5/20
525/525 - 1s - 2ms/step - accuracy: 0.8866 - loss: 0.2341 - val_accuracy: 0.9130 - val_loss: 0.1938
Epoch 6/20
525/525 - 1s - 3ms/step - accuracy: 0.8932 - loss: 0.2230 - val_accuracy: 0.9237 - val_loss: 0.1854
Epoch 7/20
525/525 - 1s - 2ms/step - accuracy: 0.9005 - loss: 0.2152 - val_accuracy: 0.9273 - val_loss: 0.1796
Epoch 8/20
525/525 - 2s - 3ms/step - accuracy: 0.9023 - loss: 0.2089 - val_accuracy: 0.9318 - val_loss: 0.1760
Epoch 9/20
525/525 - 2s - 3ms/step - accuracy: 0.9054 - loss: 0.2060 - val_accur

...

525/525 - 4s - 8ms/step - accuracy: 0.5623 - loss: 0.9928 - val_accuracy: 0.5728 - val_loss: 0.9787
Epoch 2/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9850 - val_accuracy: 0.5728 - val_loss: 0.9799
Epoch 3/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9837 - val_accuracy: 0.5728 - val_loss: 0.9790
Epoch 4/20
525/525 - 3s - 6ms/step - accuracy: 0.5689 - loss: 0.9838 - val_accuracy: 0.5728 - val_loss: 0.9789
Epoch 5/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9835 - val_accuracy: 0.5728 - val_loss: 0.9781
Epoch 6/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9836 - val_accuracy: 0.5728 - val_loss: 0.9800
Epoch 7/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9830 - val_accuracy: 0.5728 - val_loss: 0.9797
Epoch 8/20
525/525 - 2s - 4ms/step - accuracy: 0.5689 - loss: 0.9831 - val_accuracy: 0.5728 - val_loss: 0.9784
Epoch 9/20
525/525 - 2s - 5ms/step - accuracy: 0.5689 - loss: 0.9823 - val_accuracy: 0.5728 - val_loss: 0.9792
Epoch 10/20


LSTM trained in 44.99 seconds
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


525/525 - 3s - 6ms/step - accuracy: 0.5639 - loss: 0.9982 - val_accuracy: 0.5728 - val_loss: 0.9826
Epoch 2/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9873 - val_accuracy: 0.5728 - val_loss: 0.9803
Epoch 3/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9861 - val_accuracy: 0.5728 - val_loss: 0.9797
Epoch 4/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9858 - val_accuracy: 0.5728 - val_loss: 0.9798
Epoch 5/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9842 - val_accuracy: 0.5728 - val_loss: 0.9818
Epoch 6/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9829 - val_accuracy: 0.5728 - val_loss: 0.9783
Epoch 7/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9829 - val_accuracy: 0.5728 - val_loss: 0.9795
Epoch 8/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9818 - val_accuracy: 0.5728 - val_loss: 0.9793
Epoch 9/20
525/525 - 1s - 2ms/step - accuracy: 0.5689 - loss: 0.9821 - val_accuracy: 0.5728 - val_loss: 0.9784
Epoch 10/20


1D-CNN trained in 29.11 seconds
FAST VERSION: All models trained and saved successfully!
